In [4]:
import app
from app import *

app.App()

<app.App object .>

In [ ]:
from imcmodel import ImcModel
from imcview import ImcView
from imccontroller import ImcController
import tkinter as tk


class App(tk.Tk):
    def __init__(self):
        super().__init__()
        
        self.title('IMC - Formulaire de Calcul')
        
        model = ImcModel(0,0)
        
        vue = ImcView(self)
        vue.grid(row=0, column=0, padx=20, pady=20)
        
        controller = ImcController(model, vue)

        vue.set_controller(controller)

if __name__ == '__main__':
    app = App()
    app.mainloop()        
        

In [3]:
from tkinter import filedialog
import tkinter as tk

class Model(object):
    def __init__(self):
        self._args = ""
        self._selectedExcel = ""
        self.observers = []
    
    def register_observer(self, observer):
        self.observers.append(observer)
    
    def notify(self):
        [observer.update() for observer in self.observers]

    @property
    def selectedExcel(self):
        return self._selectedExcel

    @selectedExcel.setter
    def selectedExcel(self, value):
        self._selectedExcel = value
        self.notify()

class View(object):

    # The view will only be responsible for displaying widgets
    # and getting some information from user

    def __init__(self):
        # it means it arranges all widgets just as it initialize
        self.callbacks = {}
        self._showGUI("File Selection")

    def _showGUI(self, title):
        # This method will only build widgets. I prefixed it underline,
        # as it is private method of the view class, and shouldn't be
        # used outside of it (i.e. by the controller)
        self.projectWindow = tk.Tk()
        self.projectWindow.title(title)

        self.importEntry= tk.Entry(self.projectWindow, width=100)
        self.importEntry.pack(pady=10,padx=10)

        self.importButton = tk.Button(self.projectWindow, text ="Import File")
        # In this moment we don't define command of this button
        self.importButton.pack(pady=10, padx=10,side=tk.LEFT)

    def add_callback(self, key, method):
        # This method makes connections, and lets the
        # controller share its own methods to view, and prevents
        # view being dependent on controller. This method will be
        # used specifically by the controller.

        self.callbacks[key] = method

    def bind_commands(self):
        # This method also will be used by the controller. And this
        # going to happened at the moment, when callbacks will be defined

        self.importButton.config(command=self.callbacks['import'])

    def run(self):
        # This method lets program works. It also is used by the controller
        self.projectWindow.mainloop()

    def get_file_selection_view(self):
        file_path = filedialog.askopenfilename(title = "Select a Excel File", filetypes=[("Excel files", ".xlsx .xls")])
        return file_path

    def display_selection(self, content):
        self.importEntry.config(state="normal")
        self.importEntry.delete(0,'end')
        self.importEntry.insert(0,self.model.selectedElement)
        self.importEntry.config(state="readonly")
    
    
    
    
class Controller(object):
    def __init__(self, model, view):
        self.model = model
        self.view = view

        # Here we pass to the view controller method without making view
        # dependent on it
        self.view.add_callback('import', self.importButtonPressed)

        # Now, callback exist, so the controller can ask the view to bind
        # its widgets to passed values of callbacks dictionary
        self.view.bind_commands()

        self.view.run()

    def importButtonPressed(self):
        # This method, as its controller method, should lead all actions

        # First it gets file path
        selectedElement = self.view.get_file_selection_view()

        # Check if data has been selected
        if selectedElement:
            # Here it should use some model method to do whatever you expect
            # to model do, and return result
            # Honestly - I don't know what do you want from your model
            # But whatever it is let model do it itself, and return result here.
            result = self.model.some_method(selectedElement)
        else:
            pass

        # Now you can easily display result. But still, to stick with MVC
        # architecture you should use view method to got it done, instead of
        # directly using some view objects method. It should work (both view and 
        # model), like you're
        # going to make in the future another View class with same interface
        # (all public methods and parameters) and all you need to do to start using 
        # your new class, would be change imports in main.py

        self.view.display_filename(result)


#import model
#import view
#import controller


if __name__ == "__main__":
    model = Model()
    # The view doesn't need to have a controller parameter
    view = View()
    controller = Controller(model, view)